<a href="https://colab.research.google.com/github/Yodster/yodster.github.io/blob/master/Happy_Fun_Times.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Enter instance details

In [ ]:
#@title Initial Configuration { run: "auto", display-mode: "form" }
API_ID = ""#@param {type:"string"}
API_SECRET = ""#@param {type:"string"}
#@markdown The cloud tenant is the portion of the address that comes before `.cloud.extrahop.com`
CLOUD_TENANT = "extrahop-sa" #@param {type:"string"}

!rm restAPIwrapper.py
!wget https://dimarsky.net/extrahop/restAPIwrapper.py

import re
import logging
import restAPIwrapper as EH
from pprint import pprint

logger = logging.getLogger()
logging_level = "DEBUG" #@param ["INFO", "DEBUG"]
eval("logger.setLevel(logging.{0})".format(logging_level))
        
EDA = EH.ExtraHop(IPaddress = "{0}.cloud.extrahop.com".format(CLOUD_TENANT), cloud_id = API_ID, cloud_secret=API_SECRET)

##Dynamic Device Groups

In [ ]:
#@title Create Dynamic Device Group {display-mode: "form"}
#@markdown This will create a device group that inlcudes all of the specified criteria (with an or filter between them).
group_name = "Eugene Test" #@param {type: "string"}
description = "Making magic happen" #@param {type: "string"}
field_type = "name" #@param ["name", "ipaddr", "macaddr", "vendor", "tag", "activity", "node", "vlan", "discover_time", "role", "dns_name", "dhcp_name", "netbios_name", "cdp_name", "custom_name", "software", "model", "is_critical"]
#@markdown A comma separated list of criteria for the group.
criteria = "one,two" #@param {type: "string"}
criteria = criteria.split(",")
operator = "=" #@param [">", "<", "<=", ">=", "=", "!=", "startswith", "and", "or", "not", "exists", "not_exists", "~", "!~"]


post_body = {
  "description": description,
  "dynamic" : True,
    "filter": {
    "rules": [
    ],
    "operator": "or"
  },
  "name": group_name,
}


for criterion in criteria:
  post_body['filter']['rules'].append({
        "field": field_type,
        "operand": criterion.strip(),
        "operator": operator
      })
  
logging.debug("Sending payload: {0}".format(post_body))

result = EDA.postDeviceGroup(post_body)

logging.info("Received Status code of {0} and a body of {1}".format(result.status_code,result.text))
logging.info('New group ID is {0}'.format(result.headers['location'].split("/")[-1]))
